In [1]:
# 使うライブラリをインポート
import sqlite3 #データベースに使用
from bs4 import BeautifulSoup # HTMLコード解析に使用
import requests # HTMLコード取得に使用
import time # サーバーに負荷をかけないようにするために使用

In [2]:
url = 'https://www.data.jma.go.jp/obd/stats/etrn/view/monthly_s1.php?prec_no=44&block_no=47662&year=2023&month=01&day=12&view=p1' # スクレイピングするURLを定義
d_list =[] # 空のリストを作成
r = requests.get(url) # "requests"ライブラリを使用して、指定されたURLに対してHTTP GETリクエストを送信し、レスポンスを変数"r"に格納
r.encoding = 'utf-8' # 文字化けしていたため、変数"r"を'utf-8'にエンコーディング
soup = BeautifulSoup(r.text, 'html.parser') # "r.text"で変数"r"のコードを解析し、Pythonで操作しやすい形に変換する。そして"soup"に格納。"html.parser"は解析器を指定している。

# テーブル内の各行（tr要素）を取得
for row in soup.find_all('tr', class_='mtx'): # "soup"のHTMLコードから、"class_"を指定し、trタグのみを取得。それをfor文で繰り返す。
    items = row.find_all('td') # 各行内の各tdタグを取得

    if len(items) >= 9:  # 表のセルが少なくとも9つあることを確認
        cell2 = items[1].text # セル2のデータをそのまま取得
        cell5 = items[4].text # セル5のデータをそのまま取得
        cell8 = items[7].text # セル8のデータをそのまま取得
        
        # 辞書を作成
        d = {
            'pressure': cell2, # 気圧のデータ
            'temperature': cell5, # 温度のデータ
            'humidity': cell8, # 湿度のデータ
        }
        d_list.append(d) # 最後に繰り返し最初に作成したd_listに追加する

# 結果を表示
for a in d_list:
    print(a)

{'pressure': '1013.4', 'temperature': '8.0', 'humidity': '5.7'}
{'pressure': '1016.6', 'temperature': '27.5', 'humidity': '7.3'}
{'pressure': '1016.2', 'temperature': '32.0', 'humidity': '12.9'}
{'pressure': '1011.5', 'temperature': '33.0', 'humidity': '16.3'}
{'pressure': '1011.0', 'temperature': '43.5', 'humidity': '19.0'}
{'pressure': '1006.1', 'temperature': '168.5', 'humidity': '23.2'}
{'pressure': '1007.2', 'temperature': '13.5', 'humidity': '28.7'}
{'pressure': '1007.0', 'temperature': '48.5', 'humidity': '29.2'}
{'pressure': '1009.7', 'temperature': '107.5', 'humidity': '26.7'}
{'pressure': '1011.6', 'temperature': '43.5', 'humidity': '18.9'}
{'pressure': '1013.9', 'temperature': '33.5', 'humidity': '14.4'}
{'pressure': '1015.4', 'temperature': '15.5', 'humidity': '9.4'}


In [3]:
# DBファイルを保存するためのファイルパス
path = '/Users/macuser/Desktop/dsp2_work/' # ローカル（自分のMac）

# DBファイル名
db_name = 'db.sqlite'

# DBに接続する（指定したDBファイル存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)

# DBへの接続を閉じる
con.close()

In [4]:
# 1．DBに接続する
con = sqlite3.connect(path + db_name)
# print(type(con))

# 2．SQLを実行するためのオブジェクトを取得
cur = con.cursor()

# 既存のテーブルがある場合は削除する（エラー解消のため）
cur.execute('DROP TABLE IF EXISTS weather_data;')

# 3．実行したいSQLを用意する
# テーブルを作成するSQL
# CREATE TABLE テーブル名（カラム名 型，...）;
sql_create_table = 'CREATE TABLE weather_data(pressure int, temperature int, humidity int);'

# 4．SQLを実行する
cur.execute(sql_create_table)

# d_listの情報をデータベースに追加する
for item in d_list:
    cur.execute('''
        INSERT INTO weather_data (pressure, temperature, humidity)
        VALUES (?, ?, ?)
    ''', (item['pressure'], item['temperature'], item['humidity']))

# 5．必要があればコミットする（データ変更等があった場合）
# 今回は念の為コミットしておく
con.commit()

# 6．DBへの接続を閉じる
con.close()

In [5]:
# 1．DBに接続する
con = sqlite3.connect(path + db_name)

# 2．SQLを実行するためのオブジェクトを取得
cur = con.cursor()

# 3．SQLを用意
# SELECT * FROM テーブル名;
# *の部分は取得したい列の名前をカンマ区切りで指定することもできる
sql_select = 'SELECT * FROM weather_data;'

# 4．SQLを実行
cur.execute(sql_select)

# 5．結果を取得
result = cur.fetchall()

# 6．結果を表示
for row in result:
    print(row)

# 7．DBへの接続を閉じる
con.close()

(1013.4, 8, 5.7)
(1016.6, 27.5, 7.3)
(1016.2, 32, 12.9)
(1011.5, 33, 16.3)
(1011, 43.5, 19)
(1006.1, 168.5, 23.2)
(1007.2, 13.5, 28.7)
(1007, 48.5, 29.2)
(1009.7, 107.5, 26.7)
(1011.6, 43.5, 18.9)
(1013.9, 33.5, 14.4)
(1015.4, 15.5, 9.4)
